# Enter State Farm

In [8]:
from theano.sandbox import cuda
cuda.use('gpu1')

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

WARNING (theano.sandbox.cuda): Ignoring call to use(1), GPU number 0 is already in use.


In [9]:
#print (keras.__version__)
#print (theano.__version__)
#print (tensorflow.__version__)

In [10]:
import pip
for m in pip.get_installed_distributions():
    if m.project_name == 'keras':
        print(m.version)
print ("end")

end


In [11]:
#set FIXED variables
HOMEPATH = "/home/ubuntu/fastai/"
VALIDPATH = HOMEPATH + 'data/state/valid/'
TRAINPATH = HOMEPATH + 'data/state/train/'
TESTPATH = HOMEPATH + 'data/state/test/'

SAMPLEPATH = HOMEPATH + 'data/state/sample/'
SAMPLEVALIDPATH = HOMEPATH + 'data/state/sample/valid/'
SAMPLETRAINPATH = HOMEPATH + 'data/state/sample/train/'
SAMPLETESTPATH = HOMEPATH + 'data/state/sample/test/'


import os, errno

from datetime import datetime
from shutil import copyfile, move
import random

In [12]:
os.chdir(HOMEPATH)
print ("current working directory:", os.getcwd())

%matplotlib inline
from __future__ import print_function, division
#path = "data/state/"
path = "data/state/sample/"
import utils; reload(utils)
from utils import *
from IPython.display import FileLink

current working directory: /home/ubuntu/fastai


In [13]:
batch_size=64

## Create sample

The following assumes you've already created your validation set - remember that the training and validation set should contain *different drivers*, as mentioned on the Kaggle competition page.

HOLD - markdown cell  
%cd data/state  
%cd train  
%mkdir ../sample  
%mkdir ../sample/train  
%mkdir ../sample/valid  


In [7]:
#after unpacking files from kaggle data download. dirs and files already exist.

DIR = TESTPATH
print (DIR, ":", len([name for name in os.listdir(DIR) if os.path.isfile(os.path.join(DIR, name))]))
79726/10

/home/ubuntu/fastai/data/state/test/ : 79726


7972.6

In [8]:
# dir and files already exist after unpacking kaggle data.
for i in range(0, 10):
    DIR = TRAINPATH+"c"+str(i)
    print (DIR, ":", len([name for name in os.listdir(DIR) if os.path.isfile(os.path.join(DIR, name))]))

/home/ubuntu/fastai/data/state/train/c0 : 1668
/home/ubuntu/fastai/data/state/train/c1 : 1519
/home/ubuntu/fastai/data/state/train/c2 : 1553
/home/ubuntu/fastai/data/state/train/c3 : 1572
/home/ubuntu/fastai/data/state/train/c4 : 1559
/home/ubuntu/fastai/data/state/train/c5 : 1550
/home/ubuntu/fastai/data/state/train/c6 : 1558
/home/ubuntu/fastai/data/state/train/c7 : 1342
/home/ubuntu/fastai/data/state/train/c8 : 1281
/home/ubuntu/fastai/data/state/train/c9 : 1427


In [9]:
#
def makeDir(directory):
    try:
        os.makedirs(directory)
        return 1
    except OSError as e:
        if e.errno != errno.EEXIST:
            raise
            return -1
        return 0
    

In [10]:
#create dir for sample test
DIR = SAMPLETESTPATH
print (makeDir(DIR), DIR, ":", len([name for name in os.listdir(DIR) if os.path.isfile(os.path.join(DIR, name))]))

for i in range(0, 10):
    DIR = SAMPLETESTPATH+"c"+str(i)
    print (makeDir(DIR), DIR, ":", len([name for name in os.listdir(DIR) if os.path.isfile(os.path.join(DIR, name))]))

0 /home/ubuntu/fastai/data/state/sample/test/ : 0
0 /home/ubuntu/fastai/data/state/sample/test/c0 : 166
0 /home/ubuntu/fastai/data/state/sample/test/c1 : 151
0 /home/ubuntu/fastai/data/state/sample/test/c2 : 155
0 /home/ubuntu/fastai/data/state/sample/test/c3 : 157
0 /home/ubuntu/fastai/data/state/sample/test/c4 : 155
0 /home/ubuntu/fastai/data/state/sample/test/c5 : 155
0 /home/ubuntu/fastai/data/state/sample/test/c6 : 155
0 /home/ubuntu/fastai/data/state/sample/test/c7 : 134
0 /home/ubuntu/fastai/data/state/sample/test/c8 : 128
0 /home/ubuntu/fastai/data/state/sample/test/c9 : 142


In [11]:
#create dir for validation
DIR = VALIDPATH
print (makeDir(DIR), DIR, ":", len([name for name in os.listdir(DIR) if os.path.isfile(os.path.join(DIR, name))]))

for i in range(0, 10):
    DIR = VALIDPATH+"c"+str(i)
    print (makeDir(DIR), DIR, ":", len([name for name in os.listdir(DIR) if os.path.isfile(os.path.join(DIR, name))]))

0 /home/ubuntu/fastai/data/state/valid/ : 0
0 /home/ubuntu/fastai/data/state/valid/c0 : 821
0 /home/ubuntu/fastai/data/state/valid/c1 : 748
0 /home/ubuntu/fastai/data/state/valid/c2 : 764
0 /home/ubuntu/fastai/data/state/valid/c3 : 774
0 /home/ubuntu/fastai/data/state/valid/c4 : 767
0 /home/ubuntu/fastai/data/state/valid/c5 : 762
0 /home/ubuntu/fastai/data/state/valid/c6 : 767
0 /home/ubuntu/fastai/data/state/valid/c7 : 660
0 /home/ubuntu/fastai/data/state/valid/c8 : 630
0 /home/ubuntu/fastai/data/state/valid/c9 : 702


In [12]:
# move a % of the training data to validation directories.
validFract = 0.33
for i in range(0, 10):
    TRAINDIR = TRAINPATH+"c"+str(i)+"/"
    trainDirFiles = [name for name in os.listdir(TRAINDIR) if os.path.isfile(os.path.join(TRAINDIR, name))]
    numToMove = int(validFract*len(trainDirFiles))
    print (TRAINDIR, ":", len(trainDirFiles), numToMove, len(trainDirFiles)-numToMove)
    VALIDDIR = VALIDPATH+"c"+str(i)+"/"
    validDirFiles = [name for name in os.listdir(VALIDDIR) if os.path.isfile(os.path.join(VALIDDIR, name))]
    print (VALIDDIR, ":", len(validDirFiles))
    if len(validDirFiles)==0:
        #select a random validFract of files from TRAINDIR & move to VALIDDIR IFF not already done.
        moveList = random.sample(trainDirFiles, numToMove)
        print ("moving ", len(moveList))
        for file in moveList:
            #print(TRAINDIR+file, VALIDDIR+file)
            move(TRAINDIR+file, VALIDDIR+file)
            #break
    print()


/home/ubuntu/fastai/data/state/train/c0/ : 1668 550 1118
/home/ubuntu/fastai/data/state/valid/c0/ : 821

/home/ubuntu/fastai/data/state/train/c1/ : 1519 501 1018
/home/ubuntu/fastai/data/state/valid/c1/ : 748

/home/ubuntu/fastai/data/state/train/c2/ : 1553 512 1041
/home/ubuntu/fastai/data/state/valid/c2/ : 764

/home/ubuntu/fastai/data/state/train/c3/ : 1572 518 1054
/home/ubuntu/fastai/data/state/valid/c3/ : 774

/home/ubuntu/fastai/data/state/train/c4/ : 1559 514 1045
/home/ubuntu/fastai/data/state/valid/c4/ : 767

/home/ubuntu/fastai/data/state/train/c5/ : 1550 511 1039
/home/ubuntu/fastai/data/state/valid/c5/ : 762

/home/ubuntu/fastai/data/state/train/c6/ : 1558 514 1044
/home/ubuntu/fastai/data/state/valid/c6/ : 767

/home/ubuntu/fastai/data/state/train/c7/ : 1342 442 900
/home/ubuntu/fastai/data/state/valid/c7/ : 660

/home/ubuntu/fastai/data/state/train/c8/ : 1281 422 859
/home/ubuntu/fastai/data/state/valid/c8/ : 630

/home/ubuntu/fastai/data/state/train/c9/ : 1427 470 957
/

In [7]:
#create sample directories for training, validation and copy a % of files across.
#todo: fix overlap sampling error.
#select random files and copy to sample.
#select random files from sample and _MOVE_ to sample/train
#select random files from sample and MOVE to sample/valid
#move remaining files to sample/test  now have zero overlap between train/valid/test.

DIR = SAMPLEPATH
print (makeDir(DIR), DIR, ":", len([name for name in os.listdir(DIR) if os.path.isfile(os.path.join(DIR, name))]))

DIR = SAMPLEVALIDPATH 
print (makeDir(DIR), DIR, ":", len([name for name in os.listdir(DIR) if os.path.isfile(os.path.join(DIR, name))]))

DIR = SAMPLETRAINPATH
print (makeDir(DIR), DIR, ":", len([name for name in os.listdir(DIR) if os.path.isfile(os.path.join(DIR, name))]))


for i in range(0, 10):
    DIR = SAMPLEVALIDPATH+"c"+str(i)
    print (makeDir(DIR), DIR, ":", len([name for name in os.listdir(DIR) if os.path.isfile(os.path.join(DIR, name))]))


for i in range(0, 10):
    DIR = SAMPLETRAINPATH+"c"+str(i)
    print (makeDir(DIR), DIR, ":", len([name for name in os.listdir(DIR) if os.path.isfile(os.path.join(DIR, name))]))

NameError: name 'makeDir' is not defined

In [14]:
#NBBBB: __IMPORTANT___ this does not create test data correctly, potential overlap between test/train/validate.
# Need to segregate randomly selected data properly - maybe start again. UGLY algo below!!!!

sampleFract = 0.1

for i in range(0, 10):
    sampleTrainPath = SAMPLETRAINPATH+"c"+str(i)+"/"
    sampleTrainPathFiles = [name for name in os.listdir(sampleTrainPath) if os.path.isfile(os.path.join(sampleTrainPath, name))]
    print (sampleTrainPath, ":", len(sampleTrainPathFiles))
    sampleValidPath = SAMPLEVALIDPATH+"c"+str(i)+"/"
    sampleValidPathFiles = [name for name in os.listdir(sampleValidPath) if os.path.isfile(os.path.join(sampleValidPath, name))]
    print (sampleValidPath, ":", len(sampleValidPathFiles))
    sampleTestPath = SAMPLETESTPATH+"c"+str(i)+"/"
    sampleTestPathFiles = [name for name in os.listdir(sampleTestPath) if os.path.isfile(os.path.join(sampleTestPath, name))]
    print (sampleTestPath, ":", len(sampleTestPathFiles))
    
    TRAINDIR = TRAINPATH+"c"+str(i)+"/"
    trainDirFiles = [name for name in os.listdir(TRAINDIR) if os.path.isfile(os.path.join(TRAINDIR, name))]
    numToMoveTrain = int(sampleFract*len(trainDirFiles))
    print (TRAINDIR, ":", len(trainDirFiles), numToMoveTrain, len(trainDirFiles)-numToMoveTrain)

    VALIDDIR = VALIDPATH+"c"+str(i)+"/"
    validDirFiles = [name for name in os.listdir(VALIDDIR) if os.path.isfile(os.path.join(VALIDDIR, name))]
    numToMoveValid = int(sampleFract*len(validDirFiles))
    print (VALIDDIR, ":", len(validDirFiles), numToMoveValid, len(validDirFiles)-numToMoveValid)
    
    
    #now _COPY_ randomly selected files from train to sample/train    
    if len(sampleTrainPathFiles)==0:
        copyList = random.sample(trainDirFiles, numToMoveTrain)
        print ("copying ", len(copyList))
        for file in copyList:
            #print(TRAINDIR+file, sampleTrainPath+file)
            copyfile(TRAINDIR+file, sampleTrainPath+file)
            #break
    print ()
    

    #now _COPY_ randomly selected files from valid to sample/valid 
    if len(sampleValidPathFiles)==0:
        copyList = random.sample(validDirFiles, numToMoveValid)
        print ("copying ", len(copyList))
        for file in copyList:
            #print(VALIDDIR+file, sampleValidPath+file)
            copyfile(VALIDDIR+file, sampleValidPath+file)
            #break
    print ()
    #break
    


    #now _COPY_ randomly selected files from train to sample/test    
    if len(sampleTestPathFiles)==0:
        copyList = random.sample(trainDirFiles, numToMoveTrain)
        print ("copying ", len(copyList))
        for file in copyList:
            #print(TRAINDIR+file, sampleTestPath+file)
            copyfile(TRAINDIR+file, sampleTestPath+file)
            #break
    print ()

    


/home/ubuntu/fastai/data/state/sample/train/c0/ : 166
/home/ubuntu/fastai/data/state/sample/valid/c0/ : 82
/home/ubuntu/fastai/data/state/sample/test/c0/ : 166
/home/ubuntu/fastai/data/state/train/c0/ : 1668 166 1502
/home/ubuntu/fastai/data/state/valid/c0/ : 821 82 739



/home/ubuntu/fastai/data/state/sample/train/c1/ : 151
/home/ubuntu/fastai/data/state/sample/valid/c1/ : 74
/home/ubuntu/fastai/data/state/sample/test/c1/ : 151
/home/ubuntu/fastai/data/state/train/c1/ : 1519 151 1368
/home/ubuntu/fastai/data/state/valid/c1/ : 748 74 674



/home/ubuntu/fastai/data/state/sample/train/c2/ : 155
/home/ubuntu/fastai/data/state/sample/valid/c2/ : 76
/home/ubuntu/fastai/data/state/sample/test/c2/ : 155
/home/ubuntu/fastai/data/state/train/c2/ : 1553 155 1398
/home/ubuntu/fastai/data/state/valid/c2/ : 764 76 688



/home/ubuntu/fastai/data/state/sample/train/c3/ : 157
/home/ubuntu/fastai/data/state/sample/valid/c3/ : 77
/home/ubuntu/fastai/data/state/sample/test/c3/ : 157
/home/ubuntu/fasta

## Create batches

In [14]:
batches = get_batches(SAMPLETRAINPATH, batch_size=batch_size)
val_batches = get_batches(SAMPLEVALIDPATH, batch_size=batch_size*2, shuffle=False)
print (SAMPLETRAINPATH, type(batches))
print (SAMPLEVALIDPATH, type(val_batches))

#https://keras.io/preprocessing/image/
#https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/DirectoryIterator


Found 1498 images belonging to 10 classes.
Found 736 images belonging to 10 classes.
/home/ubuntu/fastai/data/state/sample/train/ <class 'keras.preprocessing.image.DirectoryIterator'>
/home/ubuntu/fastai/data/state/sample/valid/ <class 'keras.preprocessing.image.DirectoryIterator'>


In [15]:

(val_classes, trn_classes, val_labels, trn_labels, val_filenames, filenames,
    test_filename) = get_classes(SAMPLEPATH)
print ("val_classes", type(val_classes), val_classes.shape)
print ("trn_classes", type(trn_classes), trn_classes.shape)
print ("val_labels", type(val_labels), val_labels.shape)
print ("val_labels", type(val_labels), val_labels.shape)
print ("trn_labels", type(trn_labels), trn_labels.shape)
print ("val_filenames", type(val_filenames), len(val_filenames))
print ("filenames", type(filenames), len(filenames))
print ("test_filename", type(test_filename), len(test_filename))


Found 1498 images belonging to 10 classes.
Found 736 images belonging to 10 classes.
Found 1498 images belonging to 10 classes.
val_classes <type 'numpy.ndarray'> (736,)
trn_classes <type 'numpy.ndarray'> (1498,)
val_labels <type 'numpy.ndarray'> (736, 10)
val_labels <type 'numpy.ndarray'> (736, 10)
trn_labels <type 'numpy.ndarray'> (1498, 10)
val_filenames <type 'list'> 736
filenames <type 'list'> 1498
test_filename <type 'list'> 1498


## Basic models

### Linear model

First, we try the simplest model and use default parameters. Note the trick of making the first layer a batchnorm layer - that way we don't have to worry about normalizing the input ourselves.

In [16]:
model = Sequential([
        BatchNormalization(axis=1, input_shape=(3,224,224)),
        Flatten(),
        Dense(10, activation='softmax')
    ])
print (type(model))
print (len(model.layers))
for layer in model.layers:
    print (type(layer))

<class 'keras.models.Sequential'>
3
<class 'keras.layers.normalization.BatchNormalization'>
<class 'keras.layers.core.Flatten'>
<class 'keras.layers.core.Dense'>


As you can see below, this training is going nowhere...

In [17]:

startTime= datetime.now()
print ("model.compile start")
model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
print ("model.fit_generator start")
model.fit_generator(batches, batches.nb_sample, nb_epoch=2, validation_data=val_batches, 
                 nb_val_samples=val_batches.nb_sample)
timeElapsed=datetime.now()-startTime
print('Time elpased (hh:mm:ss.ms) {}'.format(timeElapsed))


model.compile start
model.fit_generator start
Epoch 1/2
1498/1498 [==============================] - 119s - loss: 13.6436 - acc: 0.1108 - val_loss: 13.7295 - val_acc: 0.1427
Epoch 2/2
1498/1498 [==============================] - 117s - loss: 13.9384 - acc: 0.1308 - val_loss: 13.5996 - val_acc: 0.1563
Time elpased (hh:mm:ss.ms) 0:03:57.080456


Let's first check the number of parameters to see that there's enough parameters to find some useful relationships:

In [19]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
batchnormalization_1 (BatchNorma (None, 3, 224, 224)   12          batchnormalization_input_1[0][0] 
____________________________________________________________________________________________________
flatten_1 (Flatten)              (None, 150528)        0           batchnormalization_1[0][0]       
____________________________________________________________________________________________________
dense_1 (Dense)                  (None, 10)            1505290     flatten_1[0][0]                  
Total params: 1,505,302
Trainable params: 1,505,296
Non-trainable params: 6
____________________________________________________________________________________________________


Over 1.5 million parameters - that should be enough. Incidentally, it's worth checking you understand why this is the number of parameters in this layer:

In [20]:
10*3*224*224
#NB: 10*3*224*224 is not an exact match to the output from model.summary

1505280

Since we have a simple model with no regularization and plenty of parameters, it seems most likely that our learning rate is too high. Perhaps it is jumping to a solution where it predicts one or two classes with high confidence, so that it can give a zero prediction to as many classes as possible - that's the best approach for a model that is no better than random, and there is likely to be where we would end up with a high learning rate. So let's check:

In [21]:
print (type(batches))

<class 'keras.preprocessing.image.DirectoryIterator'>


In [24]:
#np.round(model.predict_generator(batches, batches.N)[:10],2)
np.round(model.predict_generator(batches, batches.nb_sample)[:10],2)

#actually I got 9 sixes and one 5.

array([[ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.]], dtype=float32)

Our hypothesis was correct. It's nearly always predicting class 1 or 6, with very high confidence. So let's try a lower learning rate:

In [25]:
model = Sequential([
        BatchNormalization(axis=1, input_shape=(3,224,224)),
        Flatten(),
        Dense(10, activation='softmax')
    ])
print (len(model.layers))
for layer in model.layers:
    print (type(layer))

3
<class 'keras.layers.normalization.BatchNormalization'>
<class 'keras.layers.core.Flatten'>
<class 'keras.layers.core.Dense'>


In [26]:
startTime= datetime.now()
print ("model.compile start")
model.compile(Adam(lr=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
print ("model.fit_generator start")
model.fit_generator(batches, batches.nb_sample, nb_epoch=2, validation_data=val_batches, 
                 nb_val_samples=val_batches.nb_sample)

timeElapsed=datetime.now()-startTime
print('Time elpased (hh:mm:ss.ms) {}'.format(timeElapsed))


model.compile start
model.fit_generator start
Epoch 1/2
1498/1498 [==============================] - 119s - loss: 2.3914 - acc: 0.1856 - val_loss: 3.7583 - val_acc: 0.2079
Epoch 2/2
1498/1498 [==============================] - 117s - loss: 1.7859 - acc: 0.4012 - val_loss: 2.6118 - val_acc: 0.3261
Time elpased (hh:mm:ss.ms) 0:03:56.885882


Great - we found our way out of that hole... Now we can increase the learning rate and see where we can get to.

In [27]:
model.optimizer.lr=0.001

In [28]:
startTime= datetime.now()
print ("model.fit_generator start")
model.fit_generator(batches, batches.nb_sample, nb_epoch=4, validation_data=val_batches, 
                 nb_val_samples=val_batches.nb_sample)
timeElapsed=datetime.now()-startTime
print('Time elpased (hh:mm:ss.ms) {}'.format(timeElapsed))


model.fit_generator start
Epoch 1/4
1498/1498 [==============================] - 118s - loss: 1.3874 - acc: 0.5948 - val_loss: 2.1399 - val_acc: 0.3682
Epoch 2/4
1498/1498 [==============================] - 117s - loss: 1.1699 - acc: 0.6883 - val_loss: 1.6245 - val_acc: 0.4986
Epoch 3/4
1498/1498 [==============================] - 118s - loss: 0.9756 - acc: 0.7750 - val_loss: 1.3021 - val_acc: 0.5720
Epoch 4/4
1498/1498 [==============================] - 117s - loss: 0.8285 - acc: 0.8324 - val_loss: 1.1439 - val_acc: 0.6345
Time elpased (hh:mm:ss.ms) 0:07:51.558798


We're stabilizing at validation accuracy of 0.39. Not great, but a lot better than random. Before moving on, let's check that our validation set on the sample is large enough that it gives consistent results:

In [29]:
rnd_batches = get_batches(path+'valid', batch_size=batch_size*2, shuffle=True)

Found 736 images belonging to 10 classes.


In [30]:
val_res = [model.evaluate_generator(rnd_batches, rnd_batches.nb_sample) for i in range(10)]
np.round(val_res, 2)

array([[ 1.14,  0.63],
       [ 1.16,  0.65],
       [ 1.11,  0.64],
       [ 1.14,  0.63],
       [ 1.18,  0.62],
       [ 1.2 ,  0.6 ],
       [ 1.14,  0.63],
       [ 1.16,  0.62],
       [ 1.11,  0.66],
       [ 1.14,  0.63]])

Yup, pretty consistent - if we see improvements of 3% or more, it's probably not random, based on the above samples.

### L2 regularization

The previous model is over-fitting a lot, but we can't use dropout since we only have one layer. We can try to decrease overfitting in our model by adding [l2 regularization](http://www.kdnuggets.com/2015/04/preventing-overfitting-neural-networks.html/2) (i.e. add the sum of squares of the weights to our loss function):

In [31]:
model = Sequential([
        BatchNormalization(axis=1, input_shape=(3,224,224)),
        Flatten(),
        Dense(10, activation='softmax', W_regularizer=l2(0.01))
    ])
model.compile(Adam(lr=10e-5), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit_generator(batches, batches.nb_sample, nb_epoch=2, validation_data=val_batches, 
                 nb_val_samples=val_batches.nb_sample)

Epoch 1/2
1498/1498 [==============================] - 118s - loss: 4.3175 - acc: 0.2817 - val_loss: 10.0451 - val_acc: 0.2215
Epoch 2/2
1498/1498 [==============================] - 117s - loss: 1.5874 - acc: 0.6362 - val_loss: 2.9844 - val_acc: 0.5041


In [32]:
model.optimizer.lr=0.001

In [33]:
model.fit_generator(batches, batches.nb_sample, nb_epoch=4, validation_data=val_batches, 
                 nb_val_samples=val_batches.nb_sample)

Epoch 1/4
1498/1498 [==============================] - 117s - loss: 0.7398 - acc: 0.8371 - val_loss: 1.5246 - val_acc: 0.6780
Epoch 2/4
1498/1498 [==============================] - 117s - loss: 0.5288 - acc: 0.9045 - val_loss: 1.0557 - val_acc: 0.7758
Epoch 3/4
1498/1498 [==============================] - 117s - loss: 0.3907 - acc: 0.9539 - val_loss: 0.9057 - val_acc: 0.8043
Epoch 4/4
1498/1498 [==============================] - 117s - loss: 0.3254 - acc: 0.9740 - val_loss: 0.7415 - val_acc: 0.8370


Looks like we can get a bit over 50% accuracy this way. This will be a good benchmark for our future models - if we can't beat 50%, then we're not even beating a linear model trained on a sample, so we'll know that's not a good approach.

### Single hidden layer

The next simplest model is to add a single hidden layer.

In [34]:
model = Sequential([
        BatchNormalization(axis=1, input_shape=(3,224,224)),
        Flatten(),
        Dense(100, activation='relu'),
        BatchNormalization(),
        Dense(10, activation='softmax')
    ])
model.compile(Adam(lr=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit_generator(batches, batches.nb_sample, nb_epoch=2, validation_data=val_batches, 
                 nb_val_samples=val_batches.nb_sample)

model.optimizer.lr = 0.01
model.fit_generator(batches, batches.nb_sample, nb_epoch=5, validation_data=val_batches, 
                 nb_val_samples=val_batches.nb_sample)

Epoch 1/2
1498/1498 [==============================] - 119s - loss: 1.9473 - acc: 0.3591 - val_loss: 11.7750 - val_acc: 0.0951
Epoch 2/2
1498/1498 [==============================] - 118s - loss: 1.0643 - acc: 0.6903 - val_loss: 4.0407 - val_acc: 0.2432
Epoch 1/5
1498/1498 [==============================] - 119s - loss: 0.6261 - acc: 0.8652 - val_loss: 1.8030 - val_acc: 0.4769
Epoch 2/5
1498/1498 [==============================] - 118s - loss: 0.3892 - acc: 0.9366 - val_loss: 0.9811 - val_acc: 0.6821
Epoch 3/5
1498/1498 [==============================] - 118s - loss: 0.2719 - acc: 0.9733 - val_loss: 0.7141 - val_acc: 0.8152
Epoch 4/5
1498/1498 [==============================] - 118s - loss: 0.1955 - acc: 0.9840 - val_loss: 0.5881 - val_acc: 0.8641
Epoch 5/5
1498/1498 [==============================] - 118s - loss: 0.1343 - acc: 0.9953 - val_loss: 0.5416 - val_acc: 0.8818


Not looking very encouraging... which isn't surprising since we know that CNNs are a much better choice for computer vision problems. So we'll try one.

### Single conv layer

2 conv layers with max pooling followed by a simple dense network is a good simple CNN to start with:

In [35]:
def conv1(batches):
    model = Sequential([
            BatchNormalization(axis=1, input_shape=(3,224,224)),
            Convolution2D(32,3,3, activation='relu'),
            BatchNormalization(axis=1),
            MaxPooling2D((3,3)),
            Convolution2D(64,3,3, activation='relu'),
            BatchNormalization(axis=1),
            MaxPooling2D((3,3)),
            Flatten(),
            Dense(200, activation='relu'),
            BatchNormalization(),
            Dense(10, activation='softmax')
        ])

    model.compile(Adam(lr=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit_generator(batches, batches.nb_sample, nb_epoch=2, validation_data=val_batches, 
                     nb_val_samples=val_batches.nb_sample)
    model.optimizer.lr = 0.001
    model.fit_generator(batches, batches.nb_sample, nb_epoch=4, validation_data=val_batches, 
                     nb_val_samples=val_batches.nb_sample)
    return model

In [36]:
conv1(batches)

Epoch 1/2
1498/1498 [==============================] - 371s - loss: 1.4969 - acc: 0.5394 - val_loss: 2.1233 - val_acc: 0.3750
Epoch 2/2
1498/1498 [==============================] - 368s - loss: 0.3853 - acc: 0.9239 - val_loss: 1.5960 - val_acc: 0.4620
Epoch 1/4
1498/1498 [==============================] - 370s - loss: 0.1255 - acc: 0.9866 - val_loss: 1.8573 - val_acc: 0.4212
Epoch 2/4
1498/1498 [==============================] - 370s - loss: 0.0517 - acc: 0.9980 - val_loss: 1.9115 - val_acc: 0.4293
Epoch 3/4
1498/1498 [==============================] - 365s - loss: 0.0255 - acc: 1.0000 - val_loss: 1.9870 - val_acc: 0.4402
Epoch 4/4
1498/1498 [==============================] - 370s - loss: 0.0175 - acc: 1.0000 - val_loss: 1.9809 - val_acc: 0.4416


The training set here is very rapidly reaching a very high accuracy. So if we could regularize this, perhaps we could get a reasonable result.

So, what kind of regularization should we try first? As we discussed in lesson 3, we should start with data augmentation.

## Data augmentation

To find the best data augmentation parameters, we can try each type of data augmentation, one at a time. For each type, we can try four very different levels of augmentation, and see which is the best. In the steps below we've only kept the single best result we found. We're using the CNN we defined above, since we have already observed it can model the data quickly and accurately.

Width shift: move the image left and right -

In [37]:
gen_t = image.ImageDataGenerator(width_shift_range=0.1)
batches = get_batches(path+'train', gen_t, batch_size=batch_size)

Found 1498 images belonging to 10 classes.


In [38]:
model = conv1(batches)

Epoch 1/2
1498/1498 [==============================] - 367s - loss: 2.2033 - acc: 0.3104 - val_loss: 2.1372 - val_acc: 0.2908
Epoch 2/2
1498/1498 [==============================] - 368s - loss: 1.2707 - acc: 0.5935 - val_loss: 2.0264 - val_acc: 0.2813
Epoch 1/4
1498/1498 [==============================] - 369s - loss: 0.8686 - acc: 0.7443 - val_loss: 2.0947 - val_acc: 0.2799
Epoch 2/4
1498/1498 [==============================] - 367s - loss: 0.6418 - acc: 0.8224 - val_loss: 2.1229 - val_acc: 0.3057
Epoch 3/4
1498/1498 [==============================] - 368s - loss: 0.4906 - acc: 0.8758 - val_loss: 2.1601 - val_acc: 0.2962
Epoch 4/4
1498/1498 [==============================] - 368s - loss: 0.3687 - acc: 0.9039 - val_loss: 2.0878 - val_acc: 0.3696


Height shift: move the image up and down -

In [ ]:
gen_t = image.ImageDataGenerator(height_shift_range=0.05)
batches = get_batches(path+'train', gen_t, batch_size=batch_size)

Found 1498 images belonging to 10 classes.


In [ ]:
model = conv1(batches)

Epoch 1/2
1498/1498 [==============================] - 370s - loss: 1.9240 - acc: 0.4092 - val_loss: 1.9790 - val_acc: 0.3478
Epoch 2/2
1498/1498 [==============================] - 367s - loss: 0.8191 - acc: 0.7677 - val_loss: 2.0175 - val_acc: 0.2636
Epoch 1/4
1498/1498 [==============================] - 367s - loss: 0.4697 - acc: 0.8772 - val_loss: 2.4976 - val_acc: 0.1644
Epoch 2/4
1434/1498 [===========================>..] - ETA: 12s - loss: 0.2722 - acc: 0.9351

Random shear angles (max in radians) -

In [ ]:
gen_t = image.ImageDataGenerator(shear_range=0.1)
batches = get_batches(path+'train', gen_t, batch_size=batch_size)

In [ ]:
model = conv1(batches)

Rotation: max in degrees -

In [ ]:
gen_t = image.ImageDataGenerator(rotation_range=15)
batches = get_batches(path+'train', gen_t, batch_size=batch_size)

In [ ]:
model = conv1(batches)

Channel shift: randomly changing the R,G,B colors - 

In [ ]:
gen_t = image.ImageDataGenerator(channel_shift_range=20)
batches = get_batches(path+'train', gen_t, batch_size=batch_size)

In [ ]:
model = conv1(batches)

And finally, putting it all together!

In [ ]:
gen_t = image.ImageDataGenerator(rotation_range=15, height_shift_range=0.05, 
                shear_range=0.1, channel_shift_range=20, width_shift_range=0.1)
batches = get_batches(path+'train', gen_t, batch_size=batch_size)

In [ ]:
model = conv1(batches)

At first glance, this isn't looking encouraging, since the validation set is poor and getting worse. But the training set is getting better, and still has a long way to go in accuracy - so we should try annealing our learning rate and running more epochs, before we make a decisions.

In [ ]:
model.optimizer.lr = 0.0001
model.fit_generator(batches, batches.nb_sample, nb_epoch=5, validation_data=val_batches, 
                 nb_val_samples=val_batches.nb_sample)

Lucky we tried that - we starting to make progress! Let's keep going.

In [ ]:
model.fit_generator(batches, batches.nb_sample, nb_epoch=25, validation_data=val_batches, 
                 nb_val_samples=val_batches.nb_sample)

Amazingly, using nothing but a small sample, a simple (not pre-trained) model with no dropout, and data augmentation, we're getting results that would get us into the top 50% of the competition! This looks like a great foundation for our futher experiments.

To go further, we'll need to use the whole dataset, since dropout and data volumes are very related, so we can't tweak dropout without using all the data.